In [ ]:
import os
os.kill(os.getpid(), 9)

In [ ]:
!pip install accelerate -U

In [ ]:
import accelerate
print("Accelerate version:", accelerate.__version__)

In [ ]:
!pip install datasets

In [ ]:
!pip install arabert

In [ ]:
from datasets import load_dataset, DatasetDict , Dataset
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, Seq2SeqTrainer, Seq2SeqTrainingArguments, DataCollatorForSeq2Seq
from arabert.preprocess import ArabertPreprocessor
import torch
import os
import torch.nn as nn

In [11]:
data_path = "final_merged_summary.csv"
data = load_dataset("csv", data_files=data_path, split={'train': 'train[:90%]', 'validation': 'train[90%:]'})
print(data)

DatasetDict({
    train: Dataset({
        features: ['original', 'summary'],
        num_rows: 247747
    })
    validation: Dataset({
        features: ['original', 'summary'],
        num_rows: 27527
    })
})


In [ ]:
model_name = "malmarjeh/t5-arabic-text-summarization"
checkpoint_directory = "C:/Users/USER/Desktop/Bahaa_GP/T5_results/checkpoint-154500/checkpoint-278730/checkpoint-170335"

tokenizer = AutoTokenizer.from_pretrained(checkpoint_directory,use_fast=True)
model = AutoModelForSeq2SeqLM.from_pretrained(checkpoint_directory, ignore_mismatched_sizes=True)

preprocessor = ArabertPreprocessor(model_name=model_name)

In [ ]:
def preprocess_and_tokenize_function(examples):
    inputs = [preprocessor.preprocess(text) for text in examples['original']]
    targets = [preprocessor.preprocess(text) for text in examples['summary']]

    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length")
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(targets, max_length=128, truncation=True, padding="max_length")

    model_inputs['labels'] = [
        [(label if label != tokenizer.pad_token_id else -100) for label in label_example] for label_example in labels['input_ids']
    ]
    return model_inputs

data = data.map(preprocess_and_tokenize_function, batched=True)

In [ ]:
data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

In [ ]:
output_dir = r'C:/Users/USER/Desktop/Bahaa_GP/T5_results/checkpoint-154500/checkpoint-278730'

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir=output_dir,
    evaluation_strategy='epoch',
    save_strategy='epoch', 
    learning_rate=5e-5,
    per_device_train_batch_size=4,
    weight_decay=0.01,
    save_total_limit=10,
    num_train_epochs=11,
    predict_with_generate=True,
    fp16=torch.cuda.is_available(), 
)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=data['train'],
    eval_dataset=data['validation'],
    tokenizer=tokenizer,
    data_collator=data_collator
)


if os.path.isdir(output_dir):
    checkpoints = [os.path.join(output_dir, d) for d in os.listdir(output_dir) if d.startswith('checkpoint-')]
    if checkpoints:
        checkpoint_dir = max(checkpoints, key=os.path.getctime)
        
save_dir =r"C:\Users\USER\Desktop\Bahaa_GP\t5_weights\3"
trainer.train(resume_from_checkpoint=checkpoint_directory)

results = trainer.evaluate()
print(results)

model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)

In [ ]:
model_path = r"C:\Users\USER\Desktop\Bahaa_GP\t5_weights\3"

tokenizer = AutoTokenizer.from_pretrained(model_path)
model = AutoModelForSeq2SeqLM.from_pretrained(model_path)

In [ ]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.fc = nn.Linear(10, 2)
    
    def forward(self, x):
        return self.fc(x)


model = MyModel()


# the save path
save_dir = "C:\\Users\\USER\\Desktop\\Bahaa_GP\\saved_model"
os.makedirs(save_dir, exist_ok=True)

save_path = os.path.join(save_dir, 'model.pth')
torch.save(model.state_dict(), save_path)
print(f"Model saved to {save_path}")

In [ ]:
preprocessor = ArabertPreprocessor(model_name=model_path)

text_to_summarize = """في خطوة طموحة لتعزيز النمو الاقتصادي وتحسين جودة الحياة، أعلنت الحكومة اليوم عن إطلاق مشروع واسع النطاق لتحديث البنية التحتية في عدد من المدن الكبرى في البلاد. تشمل هذه الخطط توسيع شبكات النقل العام، تحديث الطرق والجسور، وإنشاء مناطق خضراء جديدة للسكان.

المشروع، الذي من المقرر أن يبدأ في الربع الأول من العام القادم، يهدف إلى معالجة العديد من التحديات التي تواجه المدن الكبرى، بما في ذلك الازدحام المروري، تلوث الهواء، ونقص المساحات الخضراء. وقد خصصت الحكومة ميزانية ضخمة لتمويل هذه الخطط، التي تأمل أن تجذب استثمارات إضافية من القطاع الخاص.

في تصريح لوزير التخطيط والتنمية، قال: نحن نسعى لبناء مدننا لتكون أكثر استدامة وقابلية للعيش. المشروع لن يعزز فقط البنية التحتية ولكن سيخلق أيضًا فرص عمل جديدة ويحسن البيئة العمرانية للسكان.

من المتوقع أن تشمل الأعمال التحديثية تطوير شبكات المواصلات العامة لتشمل مزيدا من الحافلات الكهربائية وتوسيع شبكات القطارات الحضرية، بالإضافة إلى تحسين البنية التحتية للطرق لتسهيل الحركة المرورية وتقليل الحوادث.

تشمل المشروعات أيضًا تجديد الأحياء القديمة وإعادة تصميم المناطق الحضرية لتشجيع الناس على استخدام الدراجات والمشي كوسائل نقل بديلة. وسيتم إنشاء المزيد من المناطق الخضراء، بما في ذلك حدائق ومسارات للمشي، مما يعزز الرفاهية العامة ويساهم في تقليل التلوث البيئي.

يتطلع المواطنون بتفاؤل إلى هذه التحسينات، حيث يعتقد الكثيرون أنها ستحدث تغييرًا كبيرًا في نوعية الحياة اليومية. ومع ذلك، هناك أيضا بعض القلق بشأن التأثيرات المحتملة على المجتمعات المحلية، خاصة فيما يتعلق بإعادة التوطين المحتملة لبعض السكان خلال عمليات التجديد.

كجزء من جهود التواصل المجتمع"""
preprocessed_text = preprocessor.preprocess(text_to_summarize)

In [ ]:
   inputs = tokenizer.encode("summarize: " + preprocessed_text, return_tensors="pt", max_length=1500, truncation=True)

In [ ]:
     outputs = model.generate(
            inputs,
            max_length=400,
            min_length=100,
            length_penalty=3.0,
            num_beams=8,
            no_repeat_ngram_size=3,
            repetition_penalty=5.0,
            early_stopping=True
        )

In [ ]:
summary = tokenizer.decode(output[0], skip_special_tokens=True)
print("Summary:", summary)